In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import json
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================
# CONFIGURATION
# ============================================

# Chemin vers le dataset complet
DATASET_PATH = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\ASL-Sensor-Dataglove-Dataset\asl_dataset_complete.csv"

# Dossier pour sauvegarder le modèle
MODEL_OUTPUT_DIR = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\ASL-Sensor-Dataglove-Dataset\model"

# Colonnes des features (capteurs)
FEATURE_COLUMNS = [
    'flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5',  # 5 Flex Sensors
    'GYRx', 'GYRy', 'GYRz',                            # 3 Gyroscope
    'ACCx', 'ACCy', 'ACCz'                             # 3 Accelerometer
]

# Paramètres du modèle
TEST_SIZE = 0.2        # 20% pour le test
RANDOM_STATE = 42      # Pour la reproductibilité
N_ESTIMATORS = 100     # Nombre d'arbres dans la forêt

# ============================================
# FONCTION 1: CHARGER ET EXPLORER LES DONNÉES
# ============================================

def load_and_explore_data():
    """
    Charge le dataset et affiche des statistiques
    """
    print("="*60)
    print("📊 CHARGEMENT DES DONNÉES")
    print("="*60)
    
    # Charger le dataset
    df = pd.read_csv(DATASET_PATH)
    
    print(f"\n✅ Dataset chargé avec succès!")
    print(f"   📏 Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"\n📋 Colonnes disponibles:")
    print(f"   {list(df.columns)}")
    
    # Informations sur les labels
    print(f"\n🏷️  LABELS (Lettres/Mots):")
    print(f"   Total de classes: {df['label'].nunique()}")
    print(f"   Classes: {sorted(df['label'].unique())}")
    
    # Répartition des données
    print(f"\n📈 RÉPARTITION PAR CLASSE:")
    label_counts = df['label'].value_counts().sort_index()
    for label, count in label_counts.items():
        print(f"   {label:15s}: {count:6d} échantillons")
    
    # Statistiques des features
    print(f"\n📊 STATISTIQUES DES FEATURES:")
    print(df[FEATURE_COLUMNS].describe())
    
    # Vérifier les valeurs manquantes
    missing = df[FEATURE_COLUMNS].isnull().sum()
    if missing.sum() > 0:
        print(f"\n⚠️  VALEURS MANQUANTES:")
        print(missing[missing > 0])
    else:
        print(f"\n✅ Aucune valeur manquante dans les features!")
    
    return df


# ============================================
# FONCTION 2: PRÉTRAITEMENT DES DONNÉES
# ============================================

def preprocess_data(df):
    """
    Prépare les données pour l'entraînement
    """
    print("\n" + "="*60)
    print("🔧 PRÉTRAITEMENT DES DONNÉES")
    print("="*60)
    
    # Extraire les features (X) et les labels (y)
    X = df[FEATURE_COLUMNS].values
    y = df['label'].values
    
    print(f"\n📊 Features (X): {X.shape}")
    print(f"🏷️  Labels (y): {y.shape}")
    
    # Encoder les labels (convertir lettres en nombres)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    print(f"\n🔢 Encodage des labels:")
    for i, label in enumerate(label_encoder.classes_[:5]):  # Afficher les 5 premiers
        print(f"   '{label}' → {i}")
    print(f"   ... ({len(label_encoder.classes_)} classes au total)")
    
    # Normaliser les features (StandardScaler)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"\n✅ Normalisation appliquée (StandardScaler)")
    print(f"   Moyenne avant: {X.mean(axis=0)[:3]} ...")
    print(f"   Moyenne après: {X_scaled.mean(axis=0)[:3]} ...")
    
    # Séparer en train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_encoded, 
        test_size=TEST_SIZE, 
        random_state=RANDOM_STATE,
        stratify=y_encoded  # Garde la même proportion de classes
    )
    
    print(f"\n📊 SÉPARATION TRAIN/TEST:")
    print(f"   🎯 Train: {X_train.shape[0]} échantillons ({(1-TEST_SIZE)*100:.0f}%)")
    print(f"   🧪 Test:  {X_test.shape[0]} échantillons ({TEST_SIZE*100:.0f}%)")
    
    return X_train, X_test, y_train, y_test, scaler, label_encoder


# ============================================
# FONCTION 3: ENTRAÎNER LE MODÈLE
# ============================================

def train_model(X_train, y_train):
    """
    Entraîne un Random Forest Classifier
    """
    print("\n" + "="*60)
    print("🚀 ENTRAÎNEMENT DU MODÈLE")
    print("="*60)
    
    # Créer le modèle
    model = RandomForestClassifier(
        n_estimators=N_ESTIMATORS,
        random_state=RANDOM_STATE,
        n_jobs=-1,  # Utiliser tous les CPU
        verbose=1
    )
    
    print(f"\n🌳 Modèle: Random Forest")
    print(f"   Nombre d'arbres: {N_ESTIMATORS}")
    print(f"   État aléatoire: {RANDOM_STATE}")
    
    # Entraîner
    print(f"\n⏳ Entraînement en cours...")
    model.fit(X_train, y_train)
    
    print(f"✅ Entraînement terminé!")
    
    # Cross-validation
    print(f"\n🔄 Validation croisée (5-fold)...")
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    print(f"   Scores: {cv_scores}")
    print(f"   Moyenne: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    
    return model


# ============================================
# FONCTION 4: ÉVALUER LE MODÈLE
# ============================================

def evaluate_model(model, X_test, y_test, label_encoder):
    """
    Évalue les performances du modèle
    """
    print("\n" + "="*60)
    print("📈 ÉVALUATION DU MODÈLE")
    print("="*60)
    
    # Prédictions
    y_pred = model.predict(X_test)
    
    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n🎯 ACCURACY: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    # Classification report
    print(f"\n📊 RAPPORT DE CLASSIFICATION:")
    print("-"*60)
    report = classification_report(
        y_test, y_pred, 
        target_names=label_encoder.classes_,
        zero_division=0
    )
    print(report)
    
    # Matrice de confusion
    print(f"\n🔲 MATRICE DE CONFUSION:")
    cm = confusion_matrix(y_test, y_pred)
    
    # Afficher seulement un aperçu si trop de classes
    if len(label_encoder.classes_) <= 10:
        print(cm)
    else:
        print(f"   (Matrice {cm.shape[0]}×{cm.shape[1]} - voir visualisation)")
    
    return accuracy, report, cm


# ============================================
# FONCTION 5: VISUALISER LES RÉSULTATS
# ============================================

def visualize_results(cm, label_encoder, feature_importance, model_dir):
    """
    Crée des visualisations des résultats
    """
    print("\n" + "="*60)
    print("📊 CRÉATION DES VISUALISATIONS")
    print("="*60)
    
    # 1. Matrice de confusion
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.title('Matrice de Confusion')
    plt.ylabel('Vraie classe')
    plt.xlabel('Classe prédite')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    cm_path = os.path.join(model_dir, 'confusion_matrix.png')
    plt.savefig(cm_path, dpi=150)
    print(f"   ✅ Matrice de confusion: {cm_path}")
    plt.close()
    
    # 2. Importance des features
    plt.figure(figsize=(10, 6))
    feature_names = FEATURE_COLUMNS
    indices = np.argsort(feature_importance)[::-1]
    
    plt.bar(range(len(feature_importance)), feature_importance[indices])
    plt.xticks(range(len(feature_importance)), 
               [feature_names[i] for i in indices], 
               rotation=45, ha='right')
    plt.title('Importance des Features')
    plt.ylabel('Importance')
    plt.xlabel('Features')
    plt.tight_layout()
    
    fi_path = os.path.join(model_dir, 'feature_importance.png')
    plt.savefig(fi_path, dpi=150)
    print(f"   ✅ Importance des features: {fi_path}")
    plt.close()


# ============================================
# FONCTION 6: SAUVEGARDER LE MODÈLE
# ============================================

def save_model(model, scaler, label_encoder, model_dir):
    """
    Sauvegarde le modèle et les transformateurs
    """
    print("\n" + "="*60)
    print("💾 SAUVEGARDE DU MODÈLE")
    print("="*60)
    
    # Créer le dossier
    os.makedirs(model_dir, exist_ok=True)
    
    # Sauvegarder le modèle
    model_path = os.path.join(model_dir, 'asl_model.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    print(f"   ✅ Modèle: {model_path}")
    
    # Sauvegarder le scaler
    scaler_path = os.path.join(model_dir, 'scaler.pkl')
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"   ✅ Scaler: {scaler_path}")
    
    # Sauvegarder le label encoder
    encoder_path = os.path.join(model_dir, 'label_encoder.pkl')
    with open(encoder_path, 'wb') as f:
        pickle.dump(label_encoder, f)
    print(f"   ✅ Label Encoder: {encoder_path}")
    
    # Sauvegarder les métadonnées en JSON
    metadata = {
        'feature_columns': FEATURE_COLUMNS,
        'classes': label_encoder.classes_.tolist(),
        'n_classes': len(label_encoder.classes_),
        'n_features': len(FEATURE_COLUMNS),
        'model_type': 'RandomForestClassifier',
        'n_estimators': N_ESTIMATORS
    }
    
    metadata_path = os.path.join(model_dir, 'model_metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=4)
    print(f"   ✅ Métadonnées: {metadata_path}")


# ============================================
# FONCTION 7: TESTER UNE PRÉDICTION
# ============================================

def test_prediction(model, scaler, label_encoder, X_test, y_test):
    """
    Teste une prédiction sur un échantillon aléatoire
    """
    print("\n" + "="*60)
    print("🧪 TEST DE PRÉDICTION")
    print("="*60)
    
    # Sélectionner un échantillon aléatoire
    idx = np.random.randint(0, len(X_test))
    sample = X_test[idx:idx+1]
    true_label = label_encoder.classes_[y_test[idx]]
    
    # Prédire
    prediction = model.predict(sample)
    predicted_label = label_encoder.classes_[prediction[0]]
    
    # Probabilités
    probabilities = model.predict_proba(sample)[0]
    
    print(f"\n📊 Échantillon #{idx}:")
    print(f"   Features: {sample[0][:5]}... (premiers 5 valeurs)")
    print(f"\n🎯 Vraie classe: {true_label}")
    print(f"🤖 Prédiction: {predicted_label}")
    print(f"   {'✅ CORRECT' if true_label == predicted_label else '❌ INCORRECT'}")
    
    print(f"\n📊 Top 3 probabilités:")
    top_3_idx = np.argsort(probabilities)[-3:][::-1]
    for i, idx in enumerate(top_3_idx, 1):
        label = label_encoder.classes_[idx]
        prob = probabilities[idx]
        print(f"   {i}. {label:15s}: {prob:.4f} ({prob*100:.2f}%)")


# ============================================
# FONCTION PRINCIPALE
# ============================================
def main():
    """
    Pipeline complet d'entraînement
    """
    print("\n" + "="*70)
    print("🤖 ENTRAÎNEMENT DU MODÈLE DE RECONNAISSANCE ASL")
    print("="*70)
   
    # 1. Charger les données
    df = load_and_explore_data()
   
    # 2. Prétraiter
    X_train, X_test, y_train, y_test, scaler, label_encoder = preprocess_data(df)
   
    # 3. Entraîner
    model = train_model(X_train, y_train)
   
    # 4. Évaluer
    accuracy, report, cm = evaluate_model(model, X_test, y_test, label_encoder)
   
    # AJOUT : Créer le dossier de sortie AVANT la visualisation
    os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
    print(f"📁 Dossier de sortie créé: {MODEL_OUTPUT_DIR}")
   
    # 5. Visualiser
    feature_importance = model.feature_importances_
    visualize_results(cm, label_encoder, feature_importance, MODEL_OUTPUT_DIR)
   
    # 6. Sauvegarder (le makedirs est redondant maintenant, mais inoffensif)
    save_model(model, scaler, label_encoder, MODEL_OUTPUT_DIR)
   
    # 7. Test de prédiction
    test_prediction(model, scaler, label_encoder, X_test, y_test)
    
    print("\n" + "="*70)
    print("✨ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
    print("="*70)
    print(f"\n📊 Résumé:")
    print(f"   🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   📁 Modèle sauvegardé dans: {MODEL_OUTPUT_DIR}")
    print(f"   🏷️  Classes reconnues: {len(label_encoder.classes_)}")
    print(f"   📊 Features utilisées: {len(FEATURE_COLUMNS)}")


# ============================================
# EXÉCUTION
# ============================================

if __name__ == "__main__":
    main()